In [15]:
import tensorflow as tf

In [20]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from keras.models import Model
import numpy as np

base_model=ResNet50(weights="imagenet")
model=Model(inputs=base_model.input, outputs=base_model.get_layer(base_model.layers[-2].name).output)
img_path="cnn_img/rabbit.jpg"
img=image.load_img(img_path, target_size=(224, 224))
x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
x=preprocess_input(x)

features=model.predict(x)
print(features.shape)
feature_layer_size=features.shape[1];

Using TensorFlow backend.


(1, 2048)


In [63]:
f16_dir="plane/f16/"
spitfire_dir="plane/spitfire/"

def create_numbered_paths(home_dir, n):
    return [home_dir+str(i)+".jpg" for i in range(n)]

def create_paired_numbered_paths(first_home_dir, second_home_dir, n):
    image_paths=[]
    for p in zip(create_numbered_paths(first_home_dir, n), create_numbered_paths(second_home_dir, n)):
        image_paths.extend(p)
    return image_paths
        
def create_features(paths, verbose=True):
    n=len(paths)
    features=np.zeros((n, feature_layer_size))
    for i in range(n):
        if (verbose==True):
            print("\t%2d / %2d"%(i+1, n))
        img=image.load_img(paths[i], target_size=(224, 224))
        img=image.img_to_array(img)
        img=np.expand_dims(img, axis=0)
        features[i, :]=preprocess_input(model.predict(img))
    
    return features
individual_n=30
image_paths=create_paired_numbered_paths(f16_dir, spitfire_dir, individual_n)

image_classes=[]
for i in range(individual_n):
    #0 stands for the pincer image and 0 stands for the scissors image
    image_classes.extend((0, 1))

#number of all images
n=60
#number of training images
n_train=30
#number of test images
n_test=n-n_train
print("Creating training features...")
#here we will store the features of training images
x_train=create_features(image_paths[:n_train])
#train classes
y_train=np.array(image_classes[:n_train])

print("Creating test features...")
#here we will store the features of test images
x_test=create_features(image_paths[n_train:])
#train classes
y_test=np.array(image_classes[n_train:])

Creating training features...
	 1 / 30
	 2 / 30
	 3 / 30
	 4 / 30
	 5 / 30
	 6 / 30
	 7 / 30
	 8 / 30
	 9 / 30
	10 / 30
	11 / 30
	12 / 30
	13 / 30
	14 / 30
	15 / 30
	16 / 30
	17 / 30
	18 / 30
	19 / 30
	20 / 30
	21 / 30
	22 / 30
	23 / 30
	24 / 30
	25 / 30
	26 / 30
	27 / 30
	28 / 30
	29 / 30
	30 / 30
Creating test features...
	 1 / 30
	 2 / 30
	 3 / 30
	 4 / 30
	 5 / 30
	 6 / 30
	 7 / 30
	 8 / 30
	 9 / 30
	10 / 30
	11 / 30
	12 / 30
	13 / 30
	14 / 30
	15 / 30
	16 / 30
	17 / 30
	18 / 30
	19 / 30
	20 / 30
	21 / 30
	22 / 30
	23 / 30
	24 / 30
	25 / 30
	26 / 30
	27 / 30
	28 / 30
	29 / 30
	30 / 30


In [64]:
print (np.shape(x_train))
print (y_train)
print (np.shape(x_test))
print (y_test)

(30, 2048)
[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
(30, 2048)
[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]


In [76]:
from sklearn import svm

def create_svm_classifier(x, y):
    #we will use linear SVM
    C=1.0
    classifier=svm.SVC(kernel="rbf", C=C);
    classifier.fit(x, y)
    return classifier

def calculate_accuracy(classifier, x, y):
    predicted=classifier.predict(x)
    return np.sum(y==predicted)/y.size

#training the model
classifier=create_svm_classifier(x_train, y_train)

#checking the model's accuracy
print("Accuracy: %.2lf%%"%(100*calculate_accuracy(classifier, x_test, y_test)))

Accuracy: 96.67%


In [74]:
for i in range(60):
    test_example_1 = np.zeros((1, feature_layer_size))
    img=image.load_img("plane/f16/"+str(i)+".jpg", target_size=(224, 224))
    img=image.img_to_array(img)
    img=np.expand_dims(img, axis=0)
    test_example_1[0,:]=preprocess_input(model.predict(img))
    print("example num "+str(i)+" " + str(classifier.predict(test_example_1)))

example num 0 [0]
example num 1 [0]
example num 2 [0]
example num 3 [0]
example num 4 [0]
example num 5 [0]
example num 6 [0]
example num 7 [0]
example num 8 [0]
example num 9 [0]
example num 10 [0]
example num 11 [0]
example num 12 [0]
example num 13 [0]
example num 14 [0]
example num 15 [0]
example num 16 [0]
example num 17 [0]
example num 18 [0]
example num 19 [0]
example num 20 [1]
example num 21 [0]
example num 22 [0]
example num 23 [0]
example num 24 [0]
example num 25 [0]
example num 26 [0]
example num 27 [0]
example num 28 [0]
example num 29 [0]
example num 30 [0]
example num 31 [0]
example num 32 [0]
example num 33 [0]
example num 34 [0]
example num 35 [0]
example num 36 [0]
example num 37 [0]
example num 38 [0]
example num 39 [0]
example num 40 [0]
example num 41 [0]
example num 42 [0]
example num 43 [0]
example num 44 [0]
example num 45 [0]
example num 46 [0]
example num 47 [0]
example num 48 [0]
example num 49 [0]
example num 50 [0]
example num 51 [0]
example num 52 [0]
exa

In [75]:
for i in range(60):
    test_example_2 = np.zeros((1,feature_layer_size))
    img = image.load_img("plane/spitfire/"+str(i)+".jpg",target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img,axis=0)
    test_example_2[0,:] = preprocess_input(model.predict(img))
    print("example num "+str(i)+" " + str(classifier.predict(test_example_2)))

example num 0 [1]
example num 1 [1]
example num 2 [1]
example num 3 [1]
example num 4 [1]
example num 5 [1]
example num 6 [1]
example num 7 [1]
example num 8 [1]
example num 9 [1]
example num 10 [1]
example num 11 [1]
example num 12 [1]
example num 13 [1]
example num 14 [1]
example num 15 [1]
example num 16 [1]
example num 17 [1]
example num 18 [1]
example num 19 [1]
example num 20 [1]
example num 21 [1]
example num 22 [1]
example num 23 [1]
example num 24 [1]
example num 25 [1]
example num 26 [1]
example num 27 [1]
example num 28 [1]
example num 29 [1]
example num 30 [1]
example num 31 [1]
example num 32 [1]
example num 33 [0]
example num 34 [1]
example num 35 [1]
example num 36 [1]
example num 37 [1]
example num 38 [1]
example num 39 [1]
example num 40 [1]
example num 41 [1]
example num 42 [1]
example num 43 [1]
example num 44 [1]
example num 45 [1]
example num 46 [1]
example num 47 [1]
example num 48 [1]
example num 49 [1]
example num 50 [1]
example num 51 [1]
example num 52 [1]
exa